In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
cd ..

/home/amiyaguchi/cs224w/wikipedia-retention


In [3]:
from pyspark.sql import SparkSession, functions as F, types as T
from src.data import import_wikipedia as impwiki


spark = impwiki.get_spark()
enwiki = impwiki.extract(spark, "data/raw/enwiki-20080103.talk.bz2")
enwiki = impwiki.transform(enwiki)

In [4]:
enwiki.cache()
enwiki.count()

13384789

In [5]:
enwiki.show(n=5, vertical=True)

-RECORD 0-----------------------------
 article_id    | 128                  
 rev_id        | 233328               
 article_title | Talk:Atlas_Shrugged  
 timestamp     | 2001-01-30 12:51:36  
 username      | ip:TimShell          
 user_id       | null                 
 category      | null                 
 image         | null                 
 main          | null                 
 talk          | null                 
 user          | null                 
 user_talk     | null                 
 other         | null                 
 external      | null                 
 template      | null                 
 comment       | *                    
 minor         | false                
 textdata      | 162                  
 year          | 2001                 
 quarter       | 1                    
-RECORD 1-----------------------------
 article_id    | 128                  
 rev_id        | 677884               
 article_title | Talk:Atlas_Shrugged  
 timestamp     | 2002-02-

In [6]:
enwiki.printSchema()

root
 |-- article_id: integer (nullable = true)
 |-- rev_id: integer (nullable = true)
 |-- article_title: string (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- username: string (nullable = true)
 |-- user_id: integer (nullable = true)
 |-- category: string (nullable = true)
 |-- image: string (nullable = true)
 |-- main: string (nullable = true)
 |-- talk: string (nullable = true)
 |-- user: string (nullable = true)
 |-- user_talk: string (nullable = true)
 |-- other: string (nullable = true)
 |-- external: string (nullable = true)
 |-- template: string (nullable = true)
 |-- comment: string (nullable = true)
 |-- minor: boolean (nullable = true)
 |-- textdata: integer (nullable = true)
 |-- year: integer (nullable = true)
 |-- quarter: integer (nullable = true)



In [7]:
enwiki.orderBy(F.rand()).show(5, truncate=False, vertical=True)

-RECORD 0-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [12]:
def with_split(dataframe):
    to_array = {"category", "image", "main", "other", "external", "template"}
    columns = [
        F.split(c, " ").alias(c) if c in to_array else c for c in dataframe.columns
    ]
    return dataframe.select(columns)

In [10]:
enwiki.orderBy(F.rand()).repartition(10).write.parquet("data/interim/test-talk-shuffled", mode="overwrite")
enwiki.orderBy("article_id", "timestamp").repartition(10).write.parquet("data/interim/test-talk-article-time", mode="overwrite")
enwiki.orderBy("timestamp").repartition(10).write.parquet("data/interim/test-talk-time", mode="overwrite")

In [13]:
enwiki_split = with_split(enwiki)
enwiki_split.orderBy(F.rand()).repartition(10).write.parquet("data/interim/test-talk-split-shuffled", mode="overwrite")
enwiki_split.orderBy("article_id", "timestamp").repartition(10).write.parquet("data/interim/test-talk-split-article-time", mode="overwrite")
enwiki_split.orderBy("timestamp").repartition(10).write.parquet("data/interim/test-talk-split-time", mode="overwrite")

In [31]:
%time enwiki_split.repartition(10).write.parquet("data/interim/test-talk-split", mode="overwrite")

CPU times: user 73.2 ms, sys: 90.7 ms, total: 164 ms
Wall time: 13min 20s


In [32]:
! tree -alh data/interim/test-talk* | grep -v .crc

data/interim/test-talk-article-time
├── [ 720M]  part-00000-24cddc3c-27b6-4153-b0b1-ad6ab994995a-c000.snappy.parquet
├── [ 720M]  part-00001-24cddc3c-27b6-4153-b0b1-ad6ab994995a-c000.snappy.parquet
├── [ 721M]  part-00002-24cddc3c-27b6-4153-b0b1-ad6ab994995a-c000.snappy.parquet
├── [ 722M]  part-00003-24cddc3c-27b6-4153-b0b1-ad6ab994995a-c000.snappy.parquet
├── [ 719M]  part-00004-24cddc3c-27b6-4153-b0b1-ad6ab994995a-c000.snappy.parquet
├── [ 720M]  part-00005-24cddc3c-27b6-4153-b0b1-ad6ab994995a-c000.snappy.parquet
├── [ 720M]  part-00006-24cddc3c-27b6-4153-b0b1-ad6ab994995a-c000.snappy.parquet
├── [ 722M]  part-00007-24cddc3c-27b6-4153-b0b1-ad6ab994995a-c000.snappy.parquet
├── [ 721M]  part-00008-24cddc3c-27b6-4153-b0b1-ad6ab994995a-c000.snappy.parquet
├── [ 720M]  part-00009-24cddc3c-27b6-4153-b0b1-ad6ab994995a-c000.snappy.parquet
├── [    0]  _SUCCESS
data/interim/test-talk-shuffled
├── [ 886M]  part-00000-ff998c16-1cd3-4c16-868b-ae1197580fc4-c000.snappy.parquet
├── [ 888M]  part-0